In [2]:

import numpy as np
import math
import sys
sys.path.append('../utils')
from itertools import combinations
from time import strftime
import pickle
import tqdm
import logging
import pymorphy2
from string import punctuation
morph = pymorphy2.MorphAnalyzer()
from nltk import TreebankWordTokenizer, word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [2]:
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

DATABASE_USER = str(os.getenv("DATABASE_USER"))
DATABASE_HOST = str(os.getenv("DATABASE_HOST"))
DATABASE_PORT = str(os.getenv("DATABASE_PORT"))
DATABASE_NAME = str(os.getenv("DATABASE_NAME"))
DATABASE_PASS = str(os.getenv("DATABASE_PASS"))
conn_string = f'postgresql://'
conn_string += f'{DATABASE_USER}:{DATABASE_PASS}'
conn_string += f'@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}'

In [3]:
from sqlalchemy import Column, Integer, VARCHAR, TIMESTAMP
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
Base = declarative_base()

In [4]:
engine = create_engine(conn_string)
Base.metadata.create_all(engine)
Session = sessionmaker(bind = engine)
session = Session()

In [11]:
# stmt = "select * from vacancy"
# session.execute(stmt).fetchall()

In [5]:
df = pd.read_sql_table('vacancy', engine)

In [6]:
df.head()

,id,created_at,updated_at,vacid,vactitle,vacdescription,vacdate,vacstatus,vaclink
0,1,2020-12-03 21:50:14.561859,2020-12-03 21:50:14.561944,337818,Мобильный менеджер по продажам,Мы предлагаем: оформление согласно трудовому ...,2020-12-03,new,https://my.sbertalents.ru/#/job-requisition/33...
1,2,2020-12-03 21:50:14.561859,2020-12-03 21:50:14.561944,453831,Специалист по прямым продажам,Мы предлагаем:трудоустройство согласно ТК РФ;г...,2020-12-03,new,https://my.sbertalents.ru/#/job-requisition/45...
2,3,2020-12-03 21:50:14.561859,2020-12-03 21:50:14.561944,618913,Менеджер по продажам,Мы предлагаем: трудоустройство согласно ТК РФ;...,2020-12-03,new,https://my.sbertalents.ru/#/job-requisition/61...
3,4,2020-12-03 21:50:14.561859,2020-12-03 21:50:14.561944,622598,Менеджер по продажам,Мы предлагаем: трудоустройство согласно ТК РФ;...,2020-12-03,new,https://my.sbertalents.ru/#/job-requisition/62...
4,5,2020-12-03 21:50:14.561859,2020-12-03 21:50:14.561944,668708,Клиентский менеджер Сбербанк Премьер,Мы предлагаем: трудоустройство согласно ТК РФ;...,2020-12-03,new,https://my.sbertalents.ru/#/job-requisition/66...


In [21]:
lines = df.vacdescription.tolist()
vacids = df.vacid.tolist()

In [8]:
def txt_pipe(lines):
 
    ru_stop_words = stopwords.words('russian')
    lines_tok = [TreebankWordTokenizer().tokenize(x) for x in lines]
    lines_tok = [[x for x in el if x not in punctuation] for el in lines_tok]
    u_norm = [[morph.parse(x)[0][2] for x in el] for el in tqdm(lines_tok)]
    u_norm = [[x for x in el if x not in ru_stop_words] for el in tqdm(u_norm)]
    corpus = [' '.join(x) for x in u_norm]
    return corpus


In [17]:
def get_tf_idf(corpus):
    tfvectorizer = TfidfVectorizer(ngram_range=(2,3))
    matrix = tfvectorizer.fit_transform(corpus)
    tfmat = matrix.toarray()
    return tfmat,tfvectorizer

In [19]:
new_corpus = ['python', 'git', 'machine learning', 'senior', 'анализировать', 'данные', 'руководить','москва','кутузов']
def get_new_tf_idf(new_corpus,tfvectorizer ):

    new_corpus = [','.join(new_corpus)]
    tf = tfvectorizer.transform(new_corpus)
    tf = tf.toarray()
    return tf

In [23]:
def get_vstacked_mats(tf,tfmat):
    return np.vstack((tf,tfmat))

In [12]:
def get_sims(new_tfmat,vacids):
    
    cossim = cosine_similarity(new_tfmat)
    cdf = pd.DataFrame(cossim)
    cdf['vacids'] = ['target'] + vacids
    cdf = cdf.set_index('vacids')
    cdf.columns = ['target'] + vacids
    scdf = cdf.loc['target'].copy()
    sorted_vacids = scdf.sort_values(ascending=False).index.tolist()[1:]
    if len(sorted_vacids) > 1:
        return sorted_vacids[1:]
    else:
        return None
    

In [13]:
corp = txt_pipe(lines)

In [18]:
tfm,vectoriser = get_tf_idf(corp)

In [20]:
tf = get_new_tf_idf(new_corpus, vectoriser)

In [24]:
stacked = get_vstacked_mats(tf,tfm)
sortedvacids = get_sims(stacked,vacids)

In [32]:
newcorpus = ['редактировать','писать','новость','статья','текст',
             'блог','вычитывать','поиск','информация','фактчекинг']

def get_sorted_vacids(newcorpus,tfm,df,vacids):
    
    tf = get_new_tf_idf(newcorpus, vectoriser, )
    stacked = get_vstacked_mats(tf,tfm)
    sortedvacids = get_sims(stacked,vacids)
    return sortedvacids

In [33]:
sortedvacids = get_sorted_vacids(newcorpus,tfm,df,vacids)

In [36]:
df[df.vacid.isin(sortedvacids[:10])][['vactitle','vacdescription']].vacdescription.tolist()

['Cбербанк приступил к реализации масштабного проекта по созданию HR-платформы – инструмента нового поколения, с помощью которого все HR сервисы будут доступны сотрудникам, руководителям, партнёрам группы компаний Сбербанк в один клик 24/7. Аналогов продукту на рынке нет, он создается с нуля БЕЗ рамок – мы даём полную свободу в выборе инструментов. Присоединяйтесь к команде профессионалов – «звёзд» IT-индустрии, которой предстоит действительно интересная и амбициозная работа, результатом которой каждый день будут пользоваться сотни тысяч человек.  Задачи:Разработка проекта с нуля - от архитектуры до документации по сопровождению;Проектирование и разработка микросервисов платформы;Построение процесса разработки и тестирования;Написание автотестов, исправление багов, участие в ревью кода; Активное участие в подборе и обучении участников команды;Взаимодействие с разработчиками, дизайнерами и другими командами в формате Agile. Наши ожидания от кандидатов:Java: Hibernate, Spring;SQL и NoSQL